In [1]:
import anatomist.api as anatomist
from soma.qt_gui.qtThread import QtThreadCall
from soma.qt_gui.qt_backend import Qt

In [2]:
from soma import aims
import json
import sys
import pandas as pd
import scipy
import scipy.stats
import numpy as np
import glob

In [3]:
a = anatomist.Anatomist()

global modules: /casa/host/build/share/anatomist-5.2/python_plugins
home   modules: /casa/home/.anatomist/python_plugins
loading module gltf_io
loading module palettecontrols
loading module measure
loading module paletteViewer
loading module meshsplit
loading module profilewindow
loading module ana_image_math
loading module anacontrolmenu
loading module foldsplit
loading module modelGraphs
loading module bsa_proba
loading module histogram
loading module gradientpalette
loading module infowindow
loading module simple_controls
loading module volumepalettes
loading module statsplotwindow
loading module save_resampled
loading module valuesplotwindow
loading module selection
all python modules loaded
Anatomist started.


In [4]:
Rspam_model = "/casa/host/build/share/brainvisa-share-5.2/models/models_2008/descriptive_models/segments/global_registered_spam_right/meshes/Rspam_model_meshes_1.arg"

Lspam_model = "/casa/host/build/share/brainvisa-share-5.2/models/models_2008/descriptive_models/segments/global_registered_spam_left/meshes/Lspam_model_meshes_1.arg"


json_regions = "/neurospin/dico/data/deep_folding/current/sulci_regions_champollion_V1.json"

path_summary = "/neurospin/dico/fred/Runs/01_betaVAE/Program/2023_jlaval_STSbabies/betaVAE/notebooks/fred/with_all_the_regions/Latent_space/With residualization"
path_file = "Proportion of predicted abnormal among patients + proportion of predicted normal among control"

In [5]:
with open(json_regions) as f:
    regions = json.load(f)

In [6]:
regions['brain']

{'S.C.-sylv._left': {'S.C._left': ['S.C._left'],
  'S.C.sylvian._left': ['S.C.sylvian._left']},
 'S.C.-S.Pe.C._left': {'S.C._left': ['S.C._left'],
  'S.C.sylvian._left': ['S.C.sylvian._left'],
  'S.Pe.C.inf._left': ['S.Pe.C.inf._left'],
  'S.Pe.C.inter._left': ['S.Pe.C.inter._left'],
  'S.Pe.C.marginal._left': ['S.Pe.C.marginal._left'],
  'S.Pe.C.sup._left': ['S.Pe.C.sup._left'],
  'S.Pe.C.median._left': ['S.Pe.C.median._left']},
 'S.C.-S.Po.C._left': {'S.C._left': ['S.C._left'],
  'S.C.sylvian._left': ['S.C.sylvian._left'],
  'S.Po.C.sup._left': ['S.Po.C.sup._left'],
  'F.I.P.Po.C.inf._left': ['F.I.P.Po.C.inf._left'],
  'F.C.L.r.retroC.tr._left': ['F.C.L.r.retroC.tr._left']},
 'S.Pe.C._left': {'S.Pe.C.inf._left': ['S.Pe.C.inf._left'],
  'S.Pe.C.inter._left': ['S.Pe.C.inter._left'],
  'S.Pe.C.marginal._left': ['S.Pe.C.marginal._left'],
  'S.Pe.C.sup._left': ['S.Pe.C.sup._left'],
  'S.Pe.C.median._left': ['S.Pe.C.median._left']},
 'S.Po.C._left': {'S.Po.C.sup._left': ['S.Po.C.sup._left'

In [7]:
len(regions['brain'])

56

In [8]:

file_to_display = f"{path_summary}/{path_file}"

In [9]:
next(iter(regions['brain']))

'S.C.-sylv._left'

In [10]:
df = pd.read_csv(file_to_display)[["region", "p"]]
#df["p"] = -np.log10(df["p"])
df.head()

,region,p
0,SCall_right,1.210526
1,SsP-SPaint_right,1.184211
2,SCall_left,1.157895
3,FPO-SCu-ScCal_left,1.131579
4,STsbr_right,1.131579


In [11]:
res = df.groupby(['region']).mean()
res["side"] = res.index.str.split('_').str[-1]
res = res.reset_index()

In [12]:
res = res[~res.region.str.contains("fronto-parietal")]
res = res[~res.region.str.contains("STs-SGSM")]
res = res[~res.region.str.contains("FCLp-SGSM")]
res = res[~res.region.str.contains("CINGULATE")]

In [13]:
"CINGULATE_left".replace("CINGULATE", "CINGULATE.")

'CINGULATE._left'

In [14]:
def get_sulci(region):
    region = region.replace("CINGULATE", "CINGULATE.")
    region = region.replace("ORBITAL", "S.Or.")
    list_sulci = list(regions['brain'][f"{region}"].keys())
    list_sulci = [x.replace("paracingular.", "S.F.int.") for x in list_sulci]
    return list_sulci

In [15]:
get_sulci("S.C.-sylv._left")

['S.C._left', 'S.C.sylvian._left']

In [16]:
dic = {
    "FCLp-subsc-FCLa-INSULA": "F.C.L.p.-subsc.-F.C.L.a.-INSULA.",
    "FCMpost-SpC": "F.C.M.post.-S.p.C.",
    "FColl-SRh": "F.Coll.-S.Rh.",
    "FIP": "F.I.P.",
    "FPO-SCu-ScCal": "F.P.O.-S.Cu.-Sc.Cal.",
    "Lobule_parietal_sup": "Lobule_parietal_sup.",
    "ORBITAL": "S.Or.",
    "SC-SPeC": "S.C.-S.Pe.C.",
    "SC-SPoC": "S.C.-S.Po.C.",
    "SC-sylv": "S.C.-sylv.",
    "SFinf-BROCA-SPeCinf": "S.F.inf.-BROCA-S.Pe.C.inf.",
    "SFint-FCMant": "S.F.int.-F.C.M.ant.",
    "SFint-SR": "S.F.int.-S.R.",
    "SFinter-SFsup": "S.F.inter.-S.F.sup.",
    "SFmarginal-SFinfant":"S.F.marginal-S.F.inf.ant.",
    "SFmedian-SFpoltr-SFsup": "S.F.median-S.F.pol.tr.-S.F.sup.",
    "SOr-SOlf": "S.Or.-S.Olf.",
    "SPeC": "S.Pe.C.",
    "SPoC": "S.Po.C.",
    "STi-SOTlat": "S.T.i.-S.O.T.lat.",
    "STi-STs-STpol": "S.T.i.-S.T.s.-S.T.pol.",
    "STsbr": "S.T.s.br.",
    "STs": "S.T.s.",
    "ScCal-SLi": "Sc.Cal.-S.Li.",
    "SsP-SPaint": "S.s.P.-S.Pa.int.",
    "FIP-FIPPoCinf": "F.I.P.-F.I.P.Po.C.inf.",
    "SCall-SsP-SintraCing": "S.Call.-S.s.P.-S.intraCing.",
    "SCall" : "S.Call.",
    "SOr" : "S.Or."
}

In [17]:
res

,region,p,side
0,FCLp-subsc-FCLa-INSULA_left,1.000000,left
1,FCLp-subsc-FCLa-INSULA_right,0.789474,right
2,FCMpost-SpC_left,0.921053,left
3,FCMpost-SpC_right,0.894737,right
4,FColl-SRh_left,0.921053,left
5,FColl-SRh_right,0.763158,right
6,FIP-FIPPoCinf_left,0.921053,left
7,FIP-FIPPoCinf_right,0.947368,right
8,FPO-SCu-ScCal_left,1.131579,left
9,FPO-SCu-ScCal_right,0.894737,right


In [18]:
import re

In [19]:
for key in dic.keys():
    res["region"] = res["region"].str.replace(key, dic[key])
res

,region,p,side
0,F.C.L.p.-subsc.-F.C.L.a.-INSULA._left,1.000000,left
1,F.C.L.p.-subsc.-F.C.L.a.-INSULA._right,0.789474,right
2,F.C.M.post.-S.p.C._left,0.921053,left
3,F.C.M.post.-S.p.C._right,0.894737,right
4,F.Coll.-S.Rh._left,0.921053,left
5,F.Coll.-S.Rh._right,0.763158,right
6,F.I.P.-F.I.P.PoCinf_left,0.921053,left
7,F.I.P.-F.I.P.PoCinf_right,0.947368,right
8,F.P.O.-S.Cu.-Sc.Cal._left,1.131579,left
9,F.P.O.-S.Cu.-Sc.Cal._right,0.894737,right


In [20]:
res.loc[6, "region"] = "F.I.P.-F.I.P.Po.C.inf._left"
res.loc[7, "region"] = "F.I.P.-F.I.P.Po.C.inf._right"
res

,region,p,side
0,F.C.L.p.-subsc.-F.C.L.a.-INSULA._left,1.000000,left
1,F.C.L.p.-subsc.-F.C.L.a.-INSULA._right,0.789474,right
2,F.C.M.post.-S.p.C._left,0.921053,left
3,F.C.M.post.-S.p.C._right,0.894737,right
4,F.Coll.-S.Rh._left,0.921053,left
5,F.Coll.-S.Rh._right,0.763158,right
6,F.I.P.-F.I.P.Po.C.inf._left,0.921053,left
7,F.I.P.-F.I.P.Po.C.inf._right,0.947368,right
8,F.P.O.-S.Cu.-Sc.Cal._left,1.131579,left
9,F.P.O.-S.Cu.-Sc.Cal._right,0.894737,right


In [21]:
res['sulcus'] = res.apply(lambda x: get_sulci(x.region), axis=1)
res.head()

,region,p,side,sulcus
0,F.C.L.p.-subsc.-F.C.L.a.-INSULA._left,1.000000,left,"[F.C.L.p._left, F.C.L.r.sc.ant._left, F.C.L.r...."
1,F.C.L.p.-subsc.-F.C.L.a.-INSULA._right,0.789474,right,"[F.C.L.p._right, F.C.L.r.sc.ant._right, F.C.L...."
2,F.C.M.post.-S.p.C._left,0.921053,left,"[F.C.M.post._left, S.C.LPC._left, S.p.C._left]"
3,F.C.M.post.-S.p.C._right,0.894737,right,"[F.C.M.post._right, S.C.LPC._right, S.p.C._right]"
4,F.Coll.-S.Rh._left,0.921053,left,"[F.Coll._left, S.Rh._left]"


In [22]:
res = res.sort_values(by="p", ascending=False)
res = res.explode("sulcus")
res[res.region.str.contains("S.T.s.")]

,region,p,side,sulcus
51,S.T.s.br._right,1.131579,right,F.I.P.r.int.1_right
51,S.T.s.br._right,1.131579,right,F.I.P.r.int.2_right
51,S.T.s.br._right,1.131579,right,S.T.s.ter.asc.ant._right
51,S.T.s.br._right,1.131579,right,S.T.s.ter.asc.post._right
51,S.T.s.br._right,1.131579,right,S.GSM._right
50,S.T.s.br._left,1.026316,left,F.I.P.r.int.1_left
50,S.T.s.br._left,1.026316,left,F.I.P.r.int.2_left
50,S.T.s.br._left,1.026316,left,S.T.s.ter.asc.ant._left
50,S.T.s.br._left,1.026316,left,S.T.s.ter.asc.post._left
50,S.T.s.br._left,1.026316,left,S.GSM._left


In [23]:
for _, row in res.iterrows():
    print(row.sulcus)

S.Call._right
S.s.P._right
S.Pa.int._right
S.Call._left
F.P.O._left
S.Pa.t._left
S.Cu._left
F.Cal.ant.-Sc.Cal._left
S.O.p._left
F.I.P.r.int.1_right
F.I.P.r.int.2_right
S.T.s.ter.asc.ant._right
S.T.s.ter.asc.post._right
S.GSM._right
F.Cal.ant.-Sc.Cal._left
S.O.p._left
S.Li.ant._left
S.Li.post._left
F.C.M.ant._left
S.F.int._left
S.Or._right
S.C._left
S.C.sylvian._left
S.Pe.C.inf._left
S.Pe.C.inter._left
S.Pe.C.marginal._left
S.Pe.C.sup._left
S.Pe.C.median._left
S.Pe.C.inf._left
S.Pe.C.inter._left
S.Pe.C.marginal._left
S.Pe.C.sup._left
S.Pe.C.median._left
S.F.marginal._left
S.F.orbitaire._left
S.F.inf.ant._left
S.F.inf._right
S.F.inf.ant._right
F.C.L.r.ant._right
F.C.L.r.asc._right
F.C.L.r.diag._right
S.Pe.C.inf._right
S.F.median._left
S.F.polaire.tr._left
S.F.sup._left
S.Call._left
S.s.P._left
S.intraCing._left
F.I.P.r.int.1_left
F.I.P.r.int.2_left
S.T.s.ter.asc.ant._left
S.T.s.ter.asc.post._left
S.GSM._left
S.F.int._right
S.R.inf._right
S.R.sup._right
S.Po.C.sup._left
F.I.P.Po.C.inf._le

In [24]:

res[res.sulcus=="S.F.orbitaire._right"]

,region,p,side,sulcus
33,S.F.marginal-S.F.inf.ant._right,0.947368,right,S.F.orbitaire._right


In [25]:

res

,region,p,side,sulcus
23,S.Call._right,1.210526,right,S.Call._right
55,S.s.P.-S.Pa.int._right,1.184211,right,S.s.P._right
55,S.s.P.-S.Pa.int._right,1.184211,right,S.Pa.int._right
22,S.Call._left,1.157895,left,S.Call._left
8,F.P.O.-S.Cu.-Sc.Cal._left,1.131579,left,F.P.O._left
...,...,...,...,...
45,S.T.i.-S.O.T.lat._right,0.631579,right,S.T.i.post._right
45,S.T.i.-S.O.T.lat._right,0.631579,right,S.O.T.lat.ant._right
45,S.T.i.-S.O.T.lat._right,0.631579,right,S.O.T.lat.int._right
45,S.T.i.-S.O.T.lat._right,0.631579,right,S.O.T.lat.med._right


In [26]:
def set_color_property(res, side):
    global dic

    if side == "L":
        spam_model_file = Lspam_model
    else:
        spam_model_file = Rspam_model
        
    dic[f"aims{side}"] = aims.read(spam_model_file)

    for vertex in dic[f"aims{side}"].vertices():
        vertex['p_value'] = 0.

    for _, row in res.iterrows():
        for vertex in dic[f"aims{side}"].vertices():
            vname = vertex.get('name')
            if vname == row.sulcus:
                    vertex['p_value'] = max(vertex['p_value'],
                                                row.p)
                    # if row.p < -np.log10(0.05/56):
                    #     vertex['p_value'] = 0.
                    # elif vertex['p_value'] != 0.: 
                    #     vertex['p_value'] = min(vertex['p_value'],
                    #                             row.p)
                    # else:
                    #     vertex['p_value'] = row.p
    
    dic[f"ana{side}"] = a.toAObject(dic[f"aims{side}"])

    dic[f"ana{side}"].setColorMode(dic[f"ana{side}"].PropertyMap)
    dic[f"ana{side}"].setColorProperty('p_value')
    dic[f"ana{side}"].notifyObservers()
    
                
def visualize_whole_hemisphere(view_quaternion, side, i):
    global block
    global dic
    try:
        block
    except NameError:
        block = a.createWindowsBlock(4)

    dic[f"win{i}"] = a.createWindow('3D',
                                    block=block,
                                    no_decoration=True,
                                    options={'hidden': 1})
    dic[f"win{i}"].addObjects(dic[f"ana{side}"])
    dic[f"ana{side}"].setPalette("green_yellow_red",
                              minVal=min(res["p"]), maxVal=max(res["p"]),
                              absoluteMode=True)
    
    dic[f"win{i}"].camera(view_quaternion=view_quaternion)

middle_view = [0.5, -0.5, -0.5, 0.5]
side_view = [0.5, 0.5, 0.5, 0.5]
bottom_view = [0, -1, 0, 0]
top_view = [0, 0, 0, -1]
 
def visualize_whole(res, side, start):
    set_color_property(res, side)
    visualize_whole_hemisphere(middle_view if side == "L" else side_view, side, start+0)
    visualize_whole_hemisphere(top_view, side, start+1)
    visualize_whole_hemisphere(bottom_view, side, start+2)
    visualize_whole_hemisphere(side_view if side == "L" else middle_view, side, start+3)

In [27]:
max(res["p"])

1.2105263157894737

In [28]:
min(res["p"])


0.631578947368421

In [29]:
dic = {}

visualize_whole(res, "L", 0)
visualize_whole(res, "R", 4)